# Spark Dataframe Operations

<ol style = "type:1">
    <li><a href = "#ref">References</a></li>
</ol>

We begin by creating a Spark session, then read `appl_stock.csv`.

In [1]:
from pyspark.sql import SparkSession

In [ ]:
# Output hidden
spark = SparkSession.builder.appName("ops").getOrCreate()

When reading `.csv` files, we have the option to infer schema. Note that this is not the can with `.json` files. We also set `header = True`, meaning that the first row of the `.csv` file is the column names.

In [3]:
df = spark.read.csv("appl_stock.csv", 
                    inferSchema = True, 
                    header = True)

In [4]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



If we want to see what the actual dataframe looks like,

In [5]:
df.show(3)

+-------------------+----------+----------+------------------+----------+---------+------------------+
|               Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+-------------------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



In [6]:
df.show(3)

+-------------------+----------+----------+------------------+----------+---------+------------------+
|               Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+-------------------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



Say we want to grab all the data that had a closing price less than \\$500. We can pass in SQL syntax:

In [7]:
df.filter("Close < 500").show(5)

+-------------------+----------+----------+------------------+------------------+---------+------------------+
|               Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
o

We can then combine it with `.select()` method, then show the result:

In [8]:
df.filter("Close < 500").select("Open").show(5)

+----------+
|      Open|
+----------+
|213.429998|
|214.599998|
|214.379993|
|    211.75|
|210.299994|
+----------+
only showing top 5 rows



To show multiple columns, pass in a list:

In [9]:
df.filter("Close < 500").select(["Open", "Close"]).show(5)

+----------+------------------+
|      Open|             Close|
+----------+------------------+
|213.429998|        214.009998|
|214.599998|        214.379993|
|214.379993|        210.969995|
|    211.75|            210.58|
|210.299994|211.98000499999998|
+----------+------------------+
only showing top 5 rows



This looks similar to SQL operators, except that you need to make sure you are calling the entire column with the dataframe, using the `df[column_name]` format. For instance,

In [11]:
df.filter(df["Close"] < 500).select("Volume").show(5)

+-------------------+----------+----------+------------------+------------------+---------+------------------+
|               Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
o

Next, let us walk through an example of filtering based off multiple conditions. Suppose we want to grab all the rows that have a closing price less than \\$200, and an open price greater than \\$200. There are two common mistakes when doing this.

**Mistake No. 1: `df.filter(df["Close"] < 200 and df["Open"] > 200).show(5)`**

We will then get an error message saying that we need to replace the `and` operator with the bitwise-and `&` operator.

**Mistake No. 2: `df.filter(df["Close"] < 200 & df["Open"] > 200).show(5)`**

This is a tricky error---we will get a `Py4JError` with insufficient cludes.

It turns out we need to distinctly separate the two conditions.

In [12]:
df.filter( (df["Close"] < 200) & (df["Open"] > 200) ).show(5)

+-------------------+------------------+----------+----------+----------+---------+------------------+
|               Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+-------------------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22 00:00:00|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28 00:00:00|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29 00:00:00|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+-------------------+------------------+----------+----------+----------+---------+------------------+



Negation is similar.

In [13]:
df.filter( (df["Close"] < 200) & ~(df["Open"] > 200) ).show(5)

+-------------------+------------------+----------+------------------+----------+---------+------------------+
|               Date|              Open|      High|               Low|     Close|   Volume|         Adj Close|
+-------------------+------------------+----------+------------------+----------+---------+------------------+
|2010-02-01 00:00:00|192.36999699999998|     196.0|191.29999899999999|194.729998|187469100|         25.229131|
|2010-02-02 00:00:00|        195.909998|196.319994|193.37999299999998|195.859997|174585600|25.375532999999997|
|2010-02-03 00:00:00|        195.169994|200.200003|        194.420004|199.229994|153832000|25.812148999999998|
|2010-02-04 00:00:00|        196.730003|198.370001|        191.570005|192.050003|189413000|         24.881912|
|2010-02-05 00:00:00|192.63000300000002|     196.0|        190.850002|195.460001|212576700|25.323710000000002|
+-------------------+------------------+----------+------------------+----------+---------+------------------+
o

**What date was the low price \\$197.16?**

In [15]:
df.filter(df["Low"] == 197.16).show()

+-------------------+------------------+----------+------+------+---------+---------+
|               Date|              Open|      High|   Low| Close|   Volume|Adj Close|
+-------------------+------------------+----------+------+------+---------+---------+
|2010-01-22 00:00:00|206.78000600000001|207.499996|197.16|197.75|220441900|25.620401|
+-------------------+------------------+----------+------+------+---------+---------+



If we use `.collect()` instead,

In [16]:
df.filter(df["Low"] == 197.16).collect()

[Row(Date=datetime.datetime(2010, 1, 22, 0, 0), Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]

then we get back a list of row objects, meaning we can save this result. In practice, collecting happens more often than showing.

In [17]:
result = df.filter(df["Low"] == 197.16).collect()

In [18]:
result[0]

Row(Date=datetime.datetime(2010, 1, 22, 0, 0), Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)

Rows can be turned into dictionaries. In fact, they have a lot of methods available to them.

In [19]:
row = result[0]

In [20]:
row.asDict()

{'Date': datetime.datetime(2010, 1, 22, 0, 0),
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

## <a name = "ref">References</a>

<ol style = "type:1">
    <li>Jose Portilla. Spark and Python for Big Data with PySpark.</li>
    <li>Apache Spark. <a href = "https://spark.apache.org/docs/latest/api/python/">https://spark.apache.org/docs/latest/api/python/</a>.</li>
</ol>